In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import config
import os.path
from os import path

from create_db import create_connection, create_table, clean_column_names
from populate_db import extract_data, insert_records
import query_db as qd

import importlib

In [2]:
importlib.reload(qd)

<module 'query_db' from '/Users/daminisharma/Dropbox/Harris MSCAPP/2019-20_Q3_Spring/Machine Learning/covid_decarceration/files/query_db.py'>

In [33]:
import datetime

In [72]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [14]:
query_court_commitment = '''
                        SELECT OFFENDER_NC_DOC_ID_NUMBER as ID, COMMITMENT_PREFIX, EARLIEST_SENTENCE_EFFECTIVE_DT, MOST_SERIOUS_OFFENSE_CODE
                        FROM OFNT3BB1 
                        WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y"
                        LIMIT 100;
                        '''

In [15]:
conn = create_connection(config.database_name)
court_commitment_example = qd.query_db_notebook(conn,query_court_commitment)
conn.close

<function Connection.close>

In [16]:
court_commitment_example

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE
0,0000004,AA,1983-07-12,None
1,0000006,AA,1973-01-30,None
2,0000006,AB,1973-04-11,None
3,0000008,AA,1990-04-09,None
4,0000008,AB,1993-08-30,None
...,...,...,...,...
95,0000135,AC,1978-03-22,None
96,0000136,AA,1992-09-15,None
97,0000142,AA,1987-03-18,None
98,0000142,AB,1988-08-01,None


In [26]:
query_sentence_computation = '''
                            SELECT INMATE_DOC_NUMBER as ID, INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, INMATE_SENTENCE_COMPONENT, max(ACTUAL_SENTENCE_END_DATE) as END_DATE
                            FROM INMT4BB1
                            GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX
                            limit 100;
                            '''

# query_sentence_computation = '''
#                             SELECT INMATE_DOC_NUMBER as ID, INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, INMATE_SENTENCE_COMPONENT, ACTUAL_SENTENCE_END_DATE
#                             FROM INMT4BB1
#                             limit 100;
#                             '''

In [27]:
conn = create_connection(config.database_name)
sentence_computation2 = qd.query_db_notebook(conn,query_sentence_computation)
conn.close

<function Connection.close>

In [31]:
sentence_computation2.tail(30)

,ID,COMMITMENT_PREFIX,INMATE_SENTENCE_COMPONENT,ACTUAL_SENTENCE_END_DATE
70,0000032,BA,014,1994-09-12
71,0000033,AA,001,1979-07-18
72,0000033,AB,001,1980-02-04
73,0000033,AC,001,1982-12-15
74,0000033,AD,001,1990-05-26
75,0000033,AD,002,1990-05-26
76,0000033,BA,001,2009-09-02
77,0000033,BA,002,2009-09-02
78,0000033,BA,003,2009-09-02
79,0000033,BA,004,2009-09-02


In [36]:

sentence_computation[sentence_computation['ID']=='0000037']

,ID,COMMITMENT_PREFIX,INMATE_SENTENCE_COMPONENT,max(ACTUAL_SENTENCE_END_DATE)
30,0000037,BA,001,2010-02-19


In [28]:
query_court_commitment = '''
                        SELECT A.OFFENDER_NC_DOC_ID_NUMBER as ID, 
                                A.COMMITMENT_PREFIX, 
                                A.EARLIEST_SENTENCE_EFFECTIVE_DT, 
                                A.MOST_SERIOUS_OFFENSE_CODE, 
                                (SELECT max(ACTUAL_SENTENCE_END_DATE)
                                    FROM INMT4BB1 B                                    
                                    WHERE A.OFFENDER_NC_DOC_ID_NUMBER = B.INMATE_DOC_NUMBER
                                    AND A.COMMITMENT_PREFIX = B.INMATE_COMMITMENT_PREFIX
                                    GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX) as END_DATE                                
                        FROM OFNT3BB1 A
                        WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y"
                        LIMIT 1000;
                        '''

In [29]:
# start = datetime.datetime.now()
# conn = create_connection(config.database_name)
# combine = qd.query_db_notebook(conn,query_court_commitment)
# conn.close
# stop = datetime.datetime.now()
# print("Time Elapsed:", stop - start) 

<function Connection.close>

In [93]:
start = datetime.datetime.now()
query_court_commitment = '''
                        SELECT A.OFFENDER_NC_DOC_ID_NUMBER as ID, 
                                A.COMMITMENT_PREFIX, 
                                A.EARLIEST_SENTENCE_EFFECTIVE_DT, 
                                A.MOST_SERIOUS_OFFENSE_CODE                              
                        FROM OFNT3BB1 A
                        WHERE NEW_PERIOD_OF_INCARCERATION_FL = "Y";
                        '''

conn = create_connection(config.database_name)
court_small = qd.query_db_notebook(conn,query_court_commitment)


query_sentence_comp = '''
                            SELECT INMATE_DOC_NUMBER as ID, INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, INMATE_COMPUTATION_STATUS_FLAG, max(ACTUAL_SENTENCE_END_DATE) as END_DATE
                            FROM INMT4BB1
                            GROUP BY INMATE_DOC_NUMBER, INMATE_COMMITMENT_PREFIX;
                        '''

sentence_compute_small = qd.query_db_notebook(conn,query_sentence_comp)

conn.close

data = court_small.merge(sentence_compute_small, on=['ID','COMMITMENT_PREFIX'], how='outer')

stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 


Time Elapsed: 0:00:38.636970


In [67]:
data.shape

(903181, 5)

In [95]:
data.groupby('INMATE_COMPUTATION_STATUS_FLAG').count()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,END_DATE
INMATE_COMPUTATION_STATUS_FLAG,,,,,
ACTIVE,27039,27039,27009,27009,27039
EAR.TERM,2970,2970,2897,2896,2970
EXPIRED,853716,853716,853022,530070,853716
FUTURE,6184,6184,6164,6164,6184
PAROLED,198,198,197,197,198
POST REL,11797,11797,11777,11777,11797
UNKNOWN,15,15,15,15,15


In [83]:
# Missing most serious offense code  - 324,308 - about a third of the data
data['MOST_SERIOUS_OFFENSE_CODE'].isnull().sum()

324308

In [96]:
# data in court commitment but not sentence computation - ~ 1,200
# I spot checked some of these and they are  ACTIVE sentences - 

data[data['END_DATE'].isnull()].head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE
114,0000156,02,1995-03-15,RIOT & CIVIL DISORDERS,NaN,NaN
287,0000327,BA,1991-05-30,None,NaN,NaN
2867,0002686,BA,1993-12-06,MISD B&E,NaN,NaN
3418,0003172,BA,1989-03-06,None,NaN,NaN
4167,0003770,BA,1995-03-25,LARCENY OF MOTOR VEHICLE,NaN,NaN


In [66]:
data[data['EARLIEST_SENTENCE_EFFECTIVE_DT'].isnull()]
# 838 observations that appear in sentence computation but not in court commitment
# i looked up a few of them and it seemed like their full record was unavailable? might be some very recent cases with a
# bit of a time lag. since there are only ~1000 in a dataset of 1 million, i'm not too worried about dropping these

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,END_DATE
902343,0001133,BA,NaN,NaN,1996-06-23
902344,0001166,AI,NaN,NaN,1990-07-06
902345,0001407,BE,NaN,NaN,2011-07-29
902346,0001640,BA,NaN,NaN,1998-03-25
902347,0005195,BF,NaN,NaN,2006-04-06
...,...,...,...,...,...
903176,1608264,BB,NaN,NaN,2022-03-04
903177,1609146,BA,NaN,NaN,2019-05-15
903178,1611526,BB,NaN,NaN,2020-02-12
903179,1615340,BA,NaN,NaN,2020-03-08


In [30]:
combine

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,END_DATE
0,0000004,AA,1983-07-12,None,1984-07-11
1,0000006,AA,1973-01-30,None,1973-03-28
2,0000006,AB,1973-04-11,None,1975-08-18
3,0000008,AA,1990-04-09,None,1990-05-17
4,0000008,AB,1993-08-30,None,1994-01-26
...,...,...,...,...,...
995,0000972,AA,1981-12-01,None,1982-08-23
996,0000972,AB,1985-07-08,None,1985-08-13
997,0000972,AC,1990-02-20,None,1990-03-20
998,0000972,AD,1991-10-28,None,1995-05-13


In [59]:
query_court_commitment_example = '''
                                    SELECT * FROM OFNT3BB1 WHERE OFFENDER_NC_DOC_ID_NUMBER = '1615340'
                                    '''

In [60]:
start = datetime.datetime.now()
conn = create_connection(config.database_name)
test = qd.query_db_notebook(conn,query_court_commitment_example)
conn.close
stop = datetime.datetime.now()
print("Time Elapsed:", stop - start) 

Time Elapsed: 0:00:07.541787


In [61]:
test

,OFFENDER_NC_DOC_ID_NUMBER,COMMITMENT_PREFIX,COMMITTED_LAST_NAME,COMMITTED_FIRST_NAME,COMMITTED_MIDDLE_NAME,COMMITTED_NAME_SUFFIX,OFFENDER_ADMISSIONINTAKE_DATE,PP_CASE_INTAKE_DATE,INMATE_COMMITMENT_STATUS_FLAG,COMMITMENT_STATUS_DATE,EARLIEST_SENTENCE_EFFECTIVE_DT,NEW_PERIOD_OF_INCARCERATION_FL,MOST_SERIOUS_OFFENSE_CODE,CO_OF_CONV_MOST_SERIOUS_OFFNSE,TOTAL_SENTENCE_LENGTH,TOTAL_JAIL_CREDITS_IN_DAYS,NO_RESTITUTION_FLAG,PP_COMMITMENT_STATUS_FLAG,PP_COMMITMENT_STATUS_DATE,TOTAL_LENGTH_OF_SUPERVISION,PED_PRIOR_TO_1995_CONVERSION,DATE_OF_LAST_UPDATE,TIME_OF_LAST_UPDATE,NEW_PERIOD_OF_SUPERVISION_FLAG,TYPE_OF_OLD_PE_DATE_CODE
0,1615340,01,CARTER,TREJAH,ELIJAH,None,0001-01-01,2019-06-12,None,0001-01-01,2019-06-12,N,OBT PROP BY FALSE PR/CHTS/SER,CRAVEN,0000000,0000,N,NORMAL NORM,2020-03-26,0010525,0001-01-01,2020-03-26,09:25:52,Y,None
1,1615340,02,CARTER,TREJAH,ELIJAH,None,0001-01-01,2019-06-12,None,0001-01-01,2019-06-12,N,B & E VEHICLES,CRAVEN,0000000,0000,N,NORMAL NORM,2020-03-26,0010525,0001-01-01,2020-03-26,09:25:52,N,None
2,1615340,03,CARTER,TREJAH,ELIJAH,None,0001-01-01,2019-06-12,None,0001-01-01,2019-07-26,N,OBT PROP BY FALSE PR/CHTS/SER,NASH,0000000,0000,N,NORMAL NORM,2020-03-26,0021120,0001-01-01,2020-03-26,09:25:52,N,None
3,1615340,04,CARTER,TREJAH,ELIJAH,None,0001-01-01,2019-06-12,None,0001-01-01,2019-07-26,N,OBT PROP BY FALSE PR/CHTS/SER,NASH,0000000,0000,N,NORMAL NORM,2020-03-26,0021120,0001-01-01,2020-03-26,09:25:52,N,None
4,1615340,05,CARTER,TREJAH,ELIJAH,None,0001-01-01,2019-06-12,None,0001-01-01,2019-07-26,N,OBT PROP BY FALSE PR/CHTS/SER,NASH,0000000,0000,N,NORMAL NORM,2020-03-26,0021120,0001-01-01,2020-03-26,09:25:52,N,None
5,1615340,06,CARTER,TREJAH,ELIJAH,None,0001-01-01,2019-06-12,None,0001-01-01,2019-08-16,N,SIMPLE ASSAULT/AFFRAY,CRAVEN,0000000,0000,N,NORMAL NORM,2020-03-26,0001200,0001-01-01,2020-03-26,09:25:52,N,None


In [38]:
query_court_commitment = '''
                        SELECT NEW_PERIOD_OF_INCARCERATION_FL, COUNT(NEW_PERIOD_OF_INCARCERATION_FL)
                        FROM OFNT3BB1
                        GROUP BY NEW_PERIOD_OF_INCARCERATION_FL;
                        '''

In [40]:
test

,NEW_PERIOD_OF_INCARCERATION_FL,COUNT(NEW_PERIOD_OF_INCARCERATION_FL)
0,None,0
1,N,2120008
2,Y,902343


In [90]:
# sanity check

query_sentence_computation = '''
                            SELECT INMATE_DOC_NUMBER as ID, INMATE_COMMITMENT_PREFIX as COMMITMENT_PREFIX, INMATE_SENTENCE_COMPONENT, ACTUAL_SENTENCE_END_DATE
                            FROM INMT4BB1
                            WHERE INMATE_DOC_NUMBER = '0002686';
                            '''

query_sentence_computation = '''
                            SELECT *
                            FROM INMT4BB1
                            WHERE INMATE_COMPUTATION_STATUS_FLAG = "ACTIVE"
                            LIMIT 1000;
                            '''


# query_sentence_computation = '''
#                             SELECT INMATE_COMPUTATION_STATUS_FLAG, COUNT(INMATE_COMPUTATION_STATUS_FLAG) 
#                             FROM INMT4BB1
#                             GROUP BY INMATE_COMPUTATION_STATUS_FLAG;
#                             '''



In [91]:
conn = create_connection(config.database_name)
sentence_computation2 = qd.query_db_notebook(conn,query_sentence_computation)
conn.close

<function Connection.close>

In [92]:

# active sentences have weird actual end dates - lots of 9999 - migth also explain why many of them are 
# missing from combined data
sentence_computation2

,INMATE_DOC_NUMBER,INMATE_COMMITMENT_PREFIX,INMATE_SENTENCE_COMPONENT,INMATE_COMPUTATION_STATUS_FLAG,SENTENCE_BEGIN_DATE_FOR_MAX,ACTUAL_SENTENCE_END_DATE,PROJECTED_RELEASE_DATE_PRD,PAROLE_DISCHARGE_DATE,PAROLE_SUPERVISION_BEGIN_DATE
0,0000080,BA,001,ACTIVE,1986-01-30,9999-01-02,9999-01-02,0001-01-01,0001-01-01
1,0000167,BA,002,ACTIVE,1998-01-21,9999-01-02,9999-01-02,0001-01-01,0001-01-01
2,0000204,BF,001,ACTIVE,2015-11-03,2022-02-10,2022-02-10,0001-01-01,0001-01-01
3,0000250,BA,001,ACTIVE,2020-01-08,2021-07-03,2021-07-03,0001-01-01,0001-01-01
4,0000289,BA,001,ACTIVE,1982-11-08,9999-01-02,9999-01-02,0001-01-01,0001-01-01
5,0000374,BB,001,ACTIVE,2016-11-07,2029-02-18,2029-02-18,0001-01-01,0001-01-01
6,0000397,BA,001,ACTIVE,1994-05-09,9999-01-02,9999-01-02,0001-01-01,0001-01-01
7,0000400,BA,001,ACTIVE,1984-05-22,9999-01-02,9999-01-02,0001-01-01,0001-01-01
8,0000411,BC,002,ACTIVE,2015-10-28,2029-09-19,2029-09-19,0001-01-01,0001-01-01
9,0000447,BF,001,ACTIVE,2009-05-12,2023-01-19,2023-01-19,0001-01-01,0001-01-01
